# 3 - Dataset preparation

In [1]:
import pandas as pd
from google.colab import drive
from pathlib import Path
import shutil
from tqdm.notebook import tqdm
from IPython.display import Audio
import librosa
import soundfile as sf

In [2]:
drive.mount('/content/drive')

songs_path = Path("/content/drive/MyDrive/Máster/DLASP/Final/songs_separated/htdemucs/")
local_data = Path("/content/data/")
local_data.mkdir(exist_ok=True)

Mounted at /content/drive


Copy all wav files with the corresponding transcription into the local machine

In [3]:
for folder in (pbar := tqdm(list(songs_path.iterdir()), leave=False)):
  pbar.set_description(folder.name)
  # shutil.copy(folder / "vocals_denoised.wav", local_data / f"{folder.name}.wav")
  shutil.copy(folder / "vocals.wav", local_data / f"{folder.name}.wav")
  shutil.copy(folder / "transcription.json", local_data / f"{folder.name}.json")

  0%|          | 0/117 [00:00<?, ?it/s]

# Manually clean


Very good samples (further speaker verification): 53, 15, 86, 90, 91, 92, 93, 95, 104, 105, 111

In [4]:
remove = [
    2, # En directo
    7, # voz femenina detras
    9,  # voz en la base
    11, # voz en la base
    16, # Mala separación base-voz
    17, # directo
    19, # mal separación 
    25, # demasiados efectos voz
    27, # mala separación base
    31, # mala separación base
    36, # voces en la base. mala sepacacion
    46, # demasiados efectos y multiples voces
    47, # voces en la base no separadas
    54, # demasiados efectos + coros
    57, # mala calidad
    59, # efectos vocales demasiado ruido
    60, # demasiados efectos + coros
    61, # mala calidad
    63, # mala separacion base
    64, # mala calidad por demasiados efectos
    67, # voces en la base
    69, # mala calidad
    71, # voces de gospel en la base
    76, # mala calidad
    79, # voces en el fondo
    83, # interludio. no voz
    89, # voces en el fondo
    96, # demasiados fragmentos con otras voces
    99, # demasiados efectos y voces
    102, # demasiado 
    106, # varios idiomas
    107, # canta con chojin, voz muy similar
    108, # canta una parte
    109, # demasiado efecto y multiples voces
    110, # demasiados efectos (eco, reverb, coros, ...)
    113, # mala separacion con la base
    115, # mala calidad
]

## Prepare dataset

In [5]:
dataset = Path('/content/dataset/')
dataset.mkdir(exist_ok=True)

In [6]:
j = 0
data = []

for song in (pbar := tqdm(list(local_data.glob('*.wav')), leave=False)):
  
  song_number = int(song.name.split("_")[0])
  if song_number in remove:
    continue
  song = song.name.split('.')[0]
  pbar.set_description(song)

  df = pd.read_json(f"/content/data/{song}.json")
  audio, sample_rate = librosa.load(f'/content/data/{song}.wav')

  for index, row in df.iterrows():
    j += 1
    start = int(sample_rate * row.start)
    end = int(sample_rate * row.end)
    filename = f"segment{j}.wav"
    segment = audio[start:end]
    sf.write(dataset / filename, segment, sample_rate)
    data.append({'text': row.text, "filename": filename, "song": song})

df_data = pd.DataFrame(data)
df_data.to_csv('/content/dataset/metadata.csv')

  0%|          | 0/117 [00:00<?, ?it/s]

Test some data

In [10]:
n = 7
row = df_data.loc[n]
y, sr = librosa.load(dataset / row.filename)
print(row.text, "\n")
Audio(y, rate=sr)

 porque haga lo que haga el premio, no cambiará mi estado de ánimo 



# Compress and upload to drive

In [8]:
shutil.make_archive("dataset", 'zip', "dataset/")

'/content/dataset.zip'

In [9]:
shutil.copy("dataset.zip", "/content/drive/MyDrive/Máster/DLASP/Final/dataset_hifi.zip")

'/content/drive/MyDrive/Máster/DLASP/Final/dataset_hifi.zip'